<a href="https://colab.research.google.com/github/AyushSrivastava27/brain_tumour_segmentation_2/blob/main/brain_tumour_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Proposed methodology
**I'm going to try new experiments in this notebook**